## Dataset

What is cpe? https://www.acunetix.com/blog/articles/common-platform-enumeration-cpe-explained/

In [5]:
import pandas as pd
import numpy as np
import json
import datasets
np.random.seed(42)

datapath = '/mnt/data/sonia/honeygan/data/processed/data.csv'
df = pd.read_csv(datapath)
print(df.isna().sum())
df.head()

os                 0
ip_str             0
port               0
module             0
cpe           734580
cpe_count          0
category           0
os_generic         0
single_cpe    824511
dtype: int64


,os,ip_str,port,module,cpe,cpe_count,category,os_generic,single_cpe
0,Ubuntu,43.205.13.243,21,ftp,NaN,0,file_sharing,ubuntu,NaN
1,Ubuntu,43.205.13.243,22,ssh,['cpe:/a:openbsd:openssh:8.2p1 Ubuntu-4ubuntu0...,1,remote_access,ubuntu,cpe:/a:openbsd:openssh:8.2p1 Ubuntu-4ubuntu0.5
2,Ubuntu,43.205.13.243,80,http,['cpe:/a:igor_sysoev:nginx'],1,webserver,ubuntu,cpe:/a:igor_sysoev:nginx
3,Ubuntu,43.205.13.243,443,https,['cpe:/a:igor_sysoev:nginx'],1,webserver,ubuntu,cpe:/a:igor_sysoev:nginx
4,Windows (Build 6.3.9600),206.233.189.205,80,http,['cpe:/a:igor_sysoev:nginx'],1,webserver,windows,cpe:/a:igor_sysoev:nginx


In [6]:
dfs = df[(~df.single_cpe.isna())]
dfs.drop(['cpe', 'cpe_count'], axis=1, inplace=True)
print(dfs.isna().sum(), df.shape)
dfs.head()

os            0
ip_str        0
port          0
module        0
category      0
os_generic    0
single_cpe    0
dtype: int64 (1043945, 9)


/tmp/ipykernel_34794/2783460533.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs.drop(['cpe', 'cpe_count'], axis=1, inplace=True)


,os,ip_str,port,module,category,os_generic,single_cpe
1,Ubuntu,43.205.13.243,22,ssh,remote_access,ubuntu,cpe:/a:openbsd:openssh:8.2p1 Ubuntu-4ubuntu0.5
2,Ubuntu,43.205.13.243,80,http,webserver,ubuntu,cpe:/a:igor_sysoev:nginx
3,Ubuntu,43.205.13.243,443,https,webserver,ubuntu,cpe:/a:igor_sysoev:nginx
4,Windows (Build 6.3.9600),206.233.189.205,80,http,webserver,windows,cpe:/a:igor_sysoev:nginx
6,Windows (Build 6.3.9600),206.233.189.205,8081,https-simple-new,webserver,windows,cpe:/a:igor_sysoev:nginx


In [7]:
zs = np.random.randint(low=0, high=9999, size=dfs.shape[0]) #latent variable

In [8]:
parsed=json.loads(dfs.to_json(orient='records'))
data_list=[{'input':str(z), 'output':str(r)} for z, r in zip(zs, parsed)]
data = datasets.Dataset.from_list(data_list)
data[:5]

{'input': ['7270', '860', '5390', '5191', '5734'],
 'output': ["{'os': 'Ubuntu', 'ip_str': '43.205.13.243', 'port': 22, 'module': 'ssh', 'category': 'remote_access', 'os_generic': 'ubuntu', 'single_cpe': 'cpe:/a:openbsd:openssh:8.2p1 Ubuntu-4ubuntu0.5'}",
  "{'os': 'Ubuntu', 'ip_str': '43.205.13.243', 'port': 80, 'module': 'http', 'category': 'webserver', 'os_generic': 'ubuntu', 'single_cpe': 'cpe:/a:igor_sysoev:nginx'}",
  "{'os': 'Ubuntu', 'ip_str': '43.205.13.243', 'port': 443, 'module': 'https', 'category': 'webserver', 'os_generic': 'ubuntu', 'single_cpe': 'cpe:/a:igor_sysoev:nginx'}",
  "{'os': 'Windows (Build 6.3.9600)', 'ip_str': '206.233.189.205', 'port': 80, 'module': 'http', 'category': 'webserver', 'os_generic': 'windows', 'single_cpe': 'cpe:/a:igor_sysoev:nginx'}",
  "{'os': 'Windows (Build 6.3.9600)', 'ip_str': '206.233.189.205', 'port': 8081, 'module': 'https-simple-new', 'category': 'webserver', 'os_generic': 'windows', 'single_cpe': 'cpe:/a:igor_sysoev:nginx'}"]}

In [10]:
sub = data[:5000]
data.save_to_disk('/mnt/data/sonia/honeygan/shodan_5000_feb14.dat')

Saving the dataset (0/1 shards):   0%|          | 0/219434 [00:00<?, ? examples/s]

## Save

In [5]:
data.save_to_disk('/mnt/data/sonia/honeygan/llama_format_feb13.dat')

Saving the dataset (0/1 shards):   0%|          | 0/219434 [00:00<?, ? examples/s]

In [6]:
reloaded_dataset = datasets.load_from_disk('/mnt/data/sonia/honeygan/llama_format_feb13.dat')
reloaded_dataset

Dataset({
    features: ['input', 'output'],
    num_rows: 219434
})